### Imports

In [1]:
import pandas as pd
import pandapower.networks as pn
from pandapower.plotting.plotly import pf_res_plotly
import data_process
import importlib

importlib.reload(data_process)

da = data_process.DataAction()
nc = data_process.net_calc()

### Initial processing

In [4]:
def initiate():
    da.folder_path = r"G:\\My Drive\\docs\\Education\\University\\ALUF\\SSE\\6 sem\\Bachelor Thesis\\data\\"
    keep_cols = ['DE_KN_residential1_grid_import',
                'DE_KN_residential2_grid_import',
                'utc_timestamp']
    da.imp_procc("house_data.csv", keep_cols)

    # prepare network
    net = pn.ieee_european_lv_asymmetric("off_peak_1440")

    # set asymmetric loads to zero
    for asy_load in net.asymmetric_load.name.tolist():
        net.asymmetric_load.p_a_mw = 0
        net.asymmetric_load.p_b_mw = 0
        net.asymmetric_load.p_c_mw = 0
        net.asymmetric_load.q_a_mvar = 0
        net.asymmetric_load.q_b_mvar = 0
        net.asymmetric_load.q_c_mvar = 0

    print("Min vm_pu bus val with no loads or sgens:", net.res_bus.vm_pu.min())
    return net

In [5]:
def iterate(net):
    """common run"""

    sgen_val = 0.010
    scaling_fac = 0.7 # without it, the network underloads
    nmbr_loads = net.asymmetric_load.shape[0]

    da.load_sgen_make(nmbr_loads)
    start_times = da.get_start_times(nmbr_loads)
    da.sgen_comm(start_times, sgen_val)
    nc.net_asym_prep(net, da.night_loads * scaling_fac, da.night_sgens)
    nc.output_writer('res_bus', 'vm_pu')
    nc.ts_run()

In [6]:
def simulate(cycles):
    # main loop
    min_vm_list = []
    min_time_list = []
    for i in range(cycles):
        while True:
            try:
                net = initiate()
                iterate(net)
                break
            except Exception as e:
                print(e)
                print("whoops...")
            
        vm_pu = nc.read_output()
        min_min_vm, min_min_ind, min_time = nc.vm_stats(vm_pu)
        min_vm_list.append(pd.Series(min_min_vm, name="min load val"))
        min_time_list.append(pd.Series(min_time, name="min load time"))

    res1 = pd.concat((min_vm_list))
    res2 = pd.concat((min_time_list))
    res = pd.concat((res1, res2), axis=1)
    res.index = range(len(res))

    return res

cycles = 10
res = simulate(cycles)
res

Number of data frame segments =  90
dfList created successfully.
Min vm_pu bus val with no loads or sgens: 1.049997179004859


100%|██████████| 721/721 [00:24<00:00, 29.40it/s] 


All-time min-load value across all busses: 1.0269
All-time min-load time across all busses: 33 , 18:33:00
Number of data frame segments =  180
dfList created successfully.
Min vm_pu bus val with no loads or sgens: 1.049997179004859
Length mismatch: Expected axis has 781 elements, new values have 721 elements
whoops...
Number of data frame segments =  270
dfList created successfully.
Min vm_pu bus val with no loads or sgens: 1.049997179004859


100%|██████████| 721/721 [00:22<00:00, 32.37it/s] 


All-time min-load value across all busses: 1.02536
All-time min-load time across all busses: 9 , 18:09:00


,min load val,min load time
0,1.02690,18:33:00
0,1.02536,18:09:00


In [14]:
path = "..\\results\\" 
name = "simulation_res.xlsx"
full = path + name
res.to_excel(full)